<a href="https://colab.research.google.com/github/Abid-Hussain36/Aviar/blob/main/aviar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing Libraries

In [ ]:
%pip install -q -U keras-tuner

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras_tuner as kt

# Loading the Data

## Fetching the Data from Kaggle

In [ ]:
! pip install kaggle

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
! cp /content/drive/MyDrive/MLFiles/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d gpiosenka/100-bird-species

Dataset URL: https://www.kaggle.com/datasets/gpiosenka/100-bird-species
License(s): CC0-1.0
100-bird-species.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
! unzip 100-bird-species.zip

Archive:  100-bird-species.zip
replace EfficientNetB0-525-(224 X 224)- 98.97.h5? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# Preprocessing and Augmenting the Data

In [ ]:
# Setting up an ImageGenerator for Data Augmentation
train_augs = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=(1./127.5 - 1), # Scales the pixel ranges to between -1 and 1
    rotation_range=40, # Randomly rotatates images up to 40 degrees,
    width_shift_range=0.2, # Randomly shifts images horizontally up to 20% of the image width
    height_shift_range=0.2, # Randomly shifts images vertically up to 20% of the image height
    shear_range=0.2, # Randomly diagonally skews the images
    zoom_range=0.2, # Randomly zooms into or out of the images
    horizontal_flip=True, # Randomly horizontally flips the images
    fill_mode="nearest"
)

# Pulls images from the train folder and randomly applies each of the transformations
train_generator = train_augs.flow_from_directory(
    "/content/train", # The directory the data is pulled from
    target_size=(224, 224), # Size of the images we want to pull
    batch_size=32, # Images grouped into batches of 32 each
    class_mode="categorical",
    shuffle=True,
    seed=42
)

valid_augs = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=(1./127.5 - 1), # Scales the pixel ranges to between -1 and 1
)

valid_generator = valid_augs.flow_from_directory(
    "/content/valid", # The directory the data is pulled from
    target_size=(224, 224), # Size of the images we want to pull
    class_mode="categorical"
)

test_augs = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=(1./127.5 - 1), # Scales the pixel ranges to between -1 and 1
)

test_generator = test_augs.flow_from_directory(
    "/content/test", # The directory the data is pulled from
    target_size=(224, 224), # Size of the images we want to pull
    class_mode="categorical"
)

In [ ]:
train_generator[0]

(array([[[[ -96.349915 ,  -87.531204 ,  -78.380394 ],
          [ -96.60064  ,  -88.03265  ,  -78.380394 ],
          [ -96.85136  ,  -88.53409  ,  -78.380394 ],
          ...,
          [-102.192154 , -104.17647  ,  -99.21568  ],
          [-102.32611  , -104.310425 ,  -99.34964  ],
          [-103.18431  , -105.168625 , -100.20784  ]],
 
         [[ -96.23921  ,  -87.30981  ,  -78.380394 ],
          [ -96.23921  ,  -87.30981  ,  -78.380394 ],
          [ -96.23921  ,  -87.30981  ,  -78.380394 ],
          ...,
          [-102.192154 , -104.17647  ,  -99.21568  ],
          [-102.58097  , -104.56528  ,  -99.6045   ],
          [-103.18431  , -105.168625 , -100.20784  ]],
 
         [[ -95.333694 ,  -86.40428  ,  -79.28592  ],
          [ -95.58441  ,  -86.655    ,  -79.035194 ],
          [ -95.83514  ,  -86.90572  ,  -78.78447  ],
          ...,
          [-102.192154 , -104.17647  ,  -99.21568  ],
          [-102.83583  , -104.820145 ,  -99.85936  ],
          [-103.18431  , -105.1

# Building and Training MobileNet Model

In [ ]:
import math

# Calculates number of mini-batches per epoch
n_steps = math.ceil(84635 / 32.0)
n_steps

### Tuning Hyperparameters for Frozen MobileNet Model

In [ ]:
def build_frozen_model(hp):
  # Sets up the value ranges for the hyperparameters
  learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-1, sampling="log")
  weight_decay = hp.Float("weight_decay", min_value=1e-4, max_value=1e-2, sampling="log")
  optimizer = hp.Choice("optimizer", values=["nesterov", "adamw"])
  if(optimizer == "nesterov"):
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9, nesterov=True)
  elif(optimizer == "adamw"):
    optimizer = tf.keras.optimizers.AdamW(learning_rate=learning_rate, weight_decay=weight_decay)

  # Constructs MobileNet model
  base_model = tf.keras.applications.MobileNet(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
  global_avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
  output = tf.keras.layers.Dense(525, activation="softmax")(global_avg)
  model = tf.keras.Model(inputs=base_model.inputs, outputs=output)

  # Freezes hidden layers of MobileNet
  for layer in base_model.layers:
    layer.trainable = False

  # Compiles the MobileNet with the selected hyperparameters
  model.compile(
      loss="categorical_crossentropy",
      optimizer=tf.keras.optimizers.SGD(),
      metrics=["accuracy"]
  )
  return model

In [ ]:
random_search_tuner_frozen = kt.RandomSearch(
    build_frozen_model,
    objective="val_accuracy",
    max_trials=3,
    overwrite=True,
    directory="bird_clf_frozen",
    project_name="rnd_search_frozen",
    seed=42
)

random_search_tuner_frozen.search(
    train_generator,
    steps_per_epoch=n_steps,
    validation_data=valid_generator,
    epochs=3
)

Trial 2 Complete [00h 54m 47s]
val_accuracy: 0.1120000034570694

Best val_accuracy So Far: 0.11809524148702621
Total elapsed time: 01h 48m 55s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
0.0044104         |0.0082848         |learning_rate
0.00013607        |0.00011943        |weight_decay
nesterov          |nesterov          |optimizer

Epoch 1/3
 404/2645 [===>..........................] - ETA: 15:00 - loss: 6.4301 - accuracy: 0.0032

### Constructing and Training the Frozen MobileNet Model with Tuned Hyperparameters

In [ ]:
base_model = tf.keras.applications.MobileNet(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
global_avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(525, activation="softmax")(global_avg)
frozen_model = tf.keras.Model(inputs=base_model.inputs, outputs=output)

for layer in base_model.layers:
  layer.trainable = False

optimizer = tf.keras.optimizers.SGD(learning_rate=0.0082848, momentum=0.9, nesterov=True)
frozen_model.compile(
    loss="categorical_crossentropy",
    optimizer=optimizer,
    metrics=["accuracy"]
)

frozen_model.fit(
    train_generator,
    steps_per_epoch=n_steps,
    validation_data=valid_generator,
    epochs=5
)

17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2645/2645 ━━━━━━━━━━━━━━━━━━━━ 1053s 394ms/step - accuracy: 0.0317 - loss: 5.9379 - val_accuracy: 0.1451 - val_loss: 4.6159
Epoch 2/5


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


2645/2645 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.1451 - val_loss: 4.6159
Epoch 3/5
2645/2645 ━━━━━━━━━━━━━━━━━━━━ 1080s 389ms/step - accuracy: 0.1147 - loss: 4.8249 - val_accuracy: 0.1962 - val_loss: 4.1823
Epoch 4/5
2645/2645 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.1962 - val_loss: 4.1823
Epoch 5/5
2645/2645 ━━━━━━━━━━━━━━━━━━━━ 1032s 389ms/step - accuracy: 0.1538 - loss: 4.4796 - val_accuracy: 0.2274 - val_loss: 3.9270


### Saving the Tuned Frozen Model for later Tuning

In [ ]:
frozen_model.save('/content/drive/MyDrive/frozen_trained_model.keras')

In [ ]:
model_frozen_check = tf.keras.models.load_model('/content/drive/MyDrive/frozen_trained_model.keras')

In [ ]:
model_frozen_check.layers

[<InputLayer name=input_layer, built=True>,
 <Conv2D name=conv1, built=True>,
 <BatchNormalization name=conv1_bn, built=True>,
 <ReLU name=conv1_relu, built=True>,
 <DepthwiseConv2D name=conv_dw_1, built=True>,
 <BatchNormalization name=conv_dw_1_bn, built=True>,
 <ReLU name=conv_dw_1_relu, built=True>,
 <Conv2D name=conv_pw_1, built=True>,
 <BatchNormalization name=conv_pw_1_bn, built=True>,
 <ReLU name=conv_pw_1_relu, built=True>,
 <ZeroPadding2D name=conv_pad_2, built=True>,
 <DepthwiseConv2D name=conv_dw_2, built=True>,
 <BatchNormalization name=conv_dw_2_bn, built=True>,
 <ReLU name=conv_dw_2_relu, built=True>,
 <Conv2D name=conv_pw_2, built=True>,
 <BatchNormalization name=conv_pw_2_bn, built=True>,
 <ReLU name=conv_pw_2_relu, built=True>,
 <DepthwiseConv2D name=conv_dw_3, built=True>,
 <BatchNormalization name=conv_dw_3_bn, built=True>,
 <ReLU name=conv_dw_3_relu, built=True>,
 <Conv2D name=conv_pw_3, built=True>,
 <BatchNormalization name=conv_pw_3_bn, built=True>,
 <ReLU name=

### Tuning Hyperparameters for Full MobileNet Model

In [ ]:
def build_full_model(hp):
  # Sets up the value ranges for the hyperparameters
  learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-1, sampling="log")
  weight_decay = hp.Float("weight_decay", min_value=1e-4, max_value=1e-2, sampling="log")
  optimizer = hp.Choice("optimizer", values=["nesterov", "adamw"])
  if(optimizer == "nesterov"):
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9, nesterov=True)
  elif(optimizer == "adamw"):
    optimizer = tf.keras.optimizers.AdamW(learning_rate=learning_rate, weight_decay=weight_decay)

  # Loads the Model with the Trained Output Layer
  model = tf.keras.models.load_model('/content/drive/MyDrive/frozen_trained_model')

  # Unfreezes the hidden layers of the MobileNet
  for layer in model.layers:
    layer.trainable = True

  # Compiles the MobileNet with the selected hyperparameters
  model.compile(
      loss="categorical_crossentropy",
      optimizer=optimizer,
      metrics=["accuracy"]
  )
  return model

In [ ]:
random_search_tuner_full = kt.RandomSearch(
    build_full_model,
    objective="val_accuracy",
    max_trials=3,
    overwrite=True,
    directory="bird_clf_full",
    project_name="rnd_search_full",
    seed=42
)

random_search_tuner_full.search(
    train_generator,
    steps_per_epoch=n_steps,
    validation_data=valid_generator,
    epochs=3
)

Trial 3 Complete [01h 02m 52s]
val_accuracy: 0.9276190400123596

Best val_accuracy So Far: 0.9276190400123596
Total elapsed time: 03h 01m 42s


In [ ]:
random_search_tuner_full.results_summary()

Results summary
Results in bird_clf_full/rnd_search_full
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
learning_rate: 0.004410354359965501
weight_decay: 0.00013607174450468629
optimizer: nesterov
Score: 0.9276190400123596

Trial 1 summary
Hyperparameters:
learning_rate: 0.0006540730319855325
weight_decay: 0.005977728042983696
optimizer: nesterov
Score: 0.9066666960716248

Trial 0 summary
Hyperparameters:
learning_rate: 0.008284768840933801
weight_decay: 0.00011942731780421024
optimizer: nesterov
Score: 0.8948571681976318


In [ ]:
random_search_tuner_full.oracle.get_best_trials(num_trials=1)[0].summary()

Trial 2 summary
Hyperparameters:
learning_rate: 0.004410354359965501
weight_decay: 0.00013607174450468629
optimizer: nesterov
Score: 0.9276190400123596


### Training and Saving the Full Model with Tuned Hyperparams

In [ ]:
full_model = tf.keras.models.load_model('/content/drive/MyDrive/frozen_trained_model.keras')

for layer in full_model.layers:
  layer.trainable = True

optimizer = tf.keras.optimizers.SGD(learning_rate=0.004410354359965501, momentum=0.9, nesterov=True)
full_model.compile(
      loss="categorical_crossentropy",
      optimizer=optimizer,
      metrics=["accuracy"]
  )

early_stopping_cb = tf.keras.callbacks.EarlyStopping(
      monitor="val_loss",
      patience=3,
      restore_best_weights=True
    )

full_model.fit(
    train_generator,
    steps_per_epoch=n_steps,
    validation_data=valid_generator,
    epochs=10,
    callbacks=[early_stopping_cb]
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2645/2645 ━━━━━━━━━━━━━━━━━━━━ 1179s 434ms/step - accuracy: 0.5267 - loss: 2.1507 - val_accuracy: 0.8735 - val_loss: 0.4910
Epoch 2/10


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


2645/2645 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8735 - val_loss: 0.4910
Epoch 3/10
2645/2645 ━━━━━━━━━━━━━━━━━━━━ 1114s 420ms/step - accuracy: 0.8400 - loss: 0.6511 - val_accuracy: 0.9078 - val_loss: 0.3387
Epoch 4/10
2645/2645 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.9078 - val_loss: 0.3387
Epoch 5/10
2645/2645 ━━━━━━━━━━━━━━━━━━━━ 1153s 419ms/step - accuracy: 0.8854 - loss: 0.4525 - val_accuracy: 0.9303 - val_loss: 0.2466
Epoch 6/10
2645/2645 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.9303 - val_loss: 0.2466
Epoch 7/10
2645/2645 ━━━━━━━━━━━━━━━━━━━━ 1105s 417ms/step - accuracy: 0.9135 - loss: 0.3409 - val_accuracy: 0.9398 - val_loss: 0.2320
Epoch 8/10
2645/2645 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.9398 - val_loss: 0.2320
Epoch 9/10
2645/2645 ━━━━━━━━━━━━━━━━━━━━ 1090s 411ms

In [ ]:
full_model.save('/content/drive/MyDrive/full_bird_clf_model.keras')

### Measuring the Full Model's Performance on the Test Set

In [ ]:
full_model.evaluate(test_generator)

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


83/83 ━━━━━━━━━━━━━━━━━━━━ 5s 59ms/step - accuracy: 0.9612 - loss: 0.1286


[0.12876902520656586, 0.961904764175415]

### Adding Supporting Preprocessing for Deployment

In [ ]:
import tensorflow as tf

def preprocess_jpg(jpg_filepath):
  # Takes in a jpg filepath and a label and preprocesses it.
  image_path = tf.io.read_file(jpg_filepath)
  image_data = tf.io.decode_jpeg(image_path, channels=3)
  image_data = tf.image.resize(image_data, [224, 224])
  image_data = (image_data / 127.5) - 1
  return image_data

In [ ]:
full_model_test = tf.keras.models.load_model('/content/drive/MyDrive/full_bird_clf_model.keras')

In [ ]:
tf.expand_dims(preprocess_jpg("/content/train/ABBOTTS BABBLER/001.jpg"), axis=0)

<tf.Tensor: shape=(1, 224, 224, 3), dtype=float32, numpy=
array([[[[ 0.85882354,  0.8745098 ,  0.7882353 ],
         [ 0.8509804 ,  0.8666667 ,  0.78039217],
         [ 0.85882354,  0.8745098 ,  0.7882353 ],
         ...,
         [-0.34117645, -0.4352941 , -0.54509807],
         [-0.40392154, -0.4980392 , -0.62352943],
         [-0.44313723, -0.5294118 , -0.67058825]],

        [[ 0.85882354,  0.8745098 ,  0.7882353 ],
         [ 0.8509804 ,  0.8666667 ,  0.78039217],
         [ 0.85882354,  0.8745098 ,  0.7882353 ],
         ...,
         [-0.3960784 , -0.49019605, -0.6       ],
         [-0.3960784 , -0.49019605, -0.6156863 ],
         [-0.38039213, -0.47450978, -0.6       ]],

        [[ 0.84313726,  0.85882354,  0.75686276],
         [ 0.84313726,  0.85882354,  0.75686276],
         [ 0.8509804 ,  0.8666667 ,  0.7647059 ],
         ...,
         [-0.38823527, -0.4823529 , -0.60784316],
         [-0.35686272, -0.45098037, -0.56078434],
         [-0.38823527, -0.4823529 , -0.5764706

In [ ]:
(full_model_test.predict(tf.expand_dims(preprocess_jpg("/content/train/ABBOTTS BABBLER/001.jpg"), axis=0))).shape

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 732ms/step


(1, 525)